# LABELS

In [1]:
import os
import re
UASpeechMLF_FilesPath=r'E:\UAspeesh dataset\mlf'
UASpeechMLF_FilesName=[file for file in os.listdir(UASpeechMLF_FilesPath)if file.endswith('.mlf')]

## Convert MLF Files To one Dict {FileName:Word}

In [2]:
allmlf_FilesWordsInDict={}
for mlf_File in UASpeechMLF_FilesName: 
    with open(UASpeechMLF_FilesPath+'\\'+mlf_File,mode='r',encoding='utf-8') as mlf_FileBuffer:
        mlf_FileBufferWords=mlf_FileBuffer.readlines()
    CounterOfmlf_FileBufferWords=0
    for line in mlf_FileBufferWords:
        CounterOfmlf_FileBufferWords+=1
        if re.match(r'"\*/\w+.lab"',line):
            allmlf_FilesWordsInDict[re.sub('"\n','',re.sub(r'"\*/','',line))]=mlf_FileBufferWords[CounterOfmlf_FileBufferWords]

## convert all mlf files to one txt file as Dict (file:word)

In [3]:
def convertDictToFile(Dict): 
    if not os.path.isdir('files'):
        os.mkdir('files')
    with open('files/labels.txt','w') as newLabelFile:
            for key in Dict:
                newLabelFile.writelines(key+':'+Dict[key])

In [4]:
convertDictToFile(allmlf_FilesWordsInDict)

## Convert MLF Files To Multi TXT Files (107100) in Folder (files)

In [ ]:
if not os.path.isdir('files'):
    os.mkdir('files')
ErrorFileNameInLineDict={}
for mlf_File in UASpeechMLF_FilesName: 
    with open(UASpeechMLF_FilesPath+'\\'+mlf_File,mode='r',encoding='utf-8') as mlf_FileBuffer:
        mlf_FileBufferWords=mlf_FileBuffer.readlines()
    CounterOfmlf_FileBufferWords=0
    for line in mlf_FileBufferWords:
        CounterOfmlf_FileBufferWords+=1
        if re.match(r'"\*/\w+.lab"',line):
            try:
                with open('files/'+re.sub('.lab"\n','',re.sub(r'"\*/','',line))+'.txt','w') as WordFile:
                    WordFile.write(mlf_FileBufferWords[CounterOfmlf_FileBufferWords])
            except:
                ErrorFileNameInLineDict[mlf_File+line]=CounterOfmlf_FileBufferWords


## Print If There Erorr OR Not 

In [ ]:
print(ErrorFileNameInLineDict)

## convert Values Of Dict Files to lists

In [5]:
print(len(allmlf_FilesWordsInDict))
# wordsInmlf_FilesInList=list(allmlf_FilesWordsInDict.values()) # use it if U Want or delete it
# audioFilesNameInList=list(allmlf_FilesWordsInDict.keys()) # use it if U Want or delete it

107100


# AUDIO

In [6]:
import os
UASpeechMainFilesOfDataPath=r'E:\UAspeesh dataset\mlf' #edit path
UASpeechMainFilesOfDataName=[folder for folder in os.listdir(UASpeechMainFilesOfDataPath)if os.path.isdir(UASpeechMainFilesOfDataPath+'\\'+folder)] #to get only Folders

In [7]:
print(UASpeechMainFilesOfDataName) # all Main Files in dataset

['CF02', 'CF03', 'CF04', 'CM04', 'CM05', 'CM06', 'CM08', 'CM09', 'CM10', 'CM12', 'CM13', 'contro', 'F02', 'F03', 'F04', 'M01', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16']


## get All Audio Files Paths

In [8]:
UASpeechDataFilesPaths=[]
for mainFile in UASpeechMainFilesOfDataName:#main Files as CF02
    for mainFileChiled in os.listdir(UASpeechMainFilesOfDataPath+'\\'+mainFile):#git records in every main file 
        if not os.path.isdir(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled):#check if it file or Folder(control)
            if mainFileChiled.endswith('.wav'):
                UASpeechDataFilesPaths.append(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled)#append Path of file .wav 
        else:
            for mainFileLeaf in os.listdir(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled): #if main folder have folder then data 
                if not os.path.isdir(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled+'\\'+mainFileLeaf):#check if it file or another Folder              
                    if mainFileLeaf.endswith('.wav'): 
                        UASpeechDataFilesPaths.append(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled+'\\'+mainFileLeaf)

In [9]:
print(UASpeechDataFilesPaths)

['E:\\UAspeesh dataset\\mlf\\CF02\\F1_S1_0001.wav', 'E:\\UAspeesh dataset\\mlf\\CF02\\F1_S1_0002.wav', 'E:\\UAspeesh dataset\\mlf\\CF02\\F1_S1_0003.wav', 'E:\\UAspeesh dataset\\mlf\\contro\\CF02\\F1_S1_0001.wav', 'E:\\UAspeesh dataset\\mlf\\contro\\CF02\\F1_S1_0002.wav', 'E:\\UAspeesh dataset\\mlf\\contro\\CF02\\F1_S1_0003.wav']


## Delete All Audio Files That Haven't Label

In [ ]:
UASpeechDataFilesPathsCopy=UASpeechDataFilesPaths.copy()
ErrorRemovedFile=[]
for audioFilePath in UASpeechDataFilesPaths:
    audioFileName=re.sub('.wav','.lab',audioFilePath.split('\\')[-1])
    if audioFileName not in allmlf_FilesWordsInDict:
        try:
            os.remove(audioFilePath)
            UASpeechDataFilesPathsCopy.remove(audioFilePath)
        except:
            ErrorRemovedFile.append(audioFilePath)
UASpeechDataFilesPaths=UASpeechDataFilesPathsCopy.copy()
del(UASpeechDataFilesPathsCopy)

In [ ]:
print(UASpeechDataFilesPaths) #all paths after update
print(ErrorRemovedFile) #if there file have error and didn't remove

## Delete All labels That Haven't Audio File

In [ ]:
UASpeechDataFilesNames=[re.sub('.wav','.lab',audioFileName.split('\\')[-1]) for audioFileName in UASpeechDataFilesPaths ]
allmlf_FilesWordsInDictCopy=allmlf_FilesWordsInDict.copy()
for mlf_FileWord in allmlf_FilesWordsInDict:
    if mlf_FileWord not in UASpeechDataFilesNames:
        allmlf_FilesWordsInDictCopy.pop(mlf_FileWord)
allmlf_FilesWordsInDict=allmlf_FilesWordsInDictCopy.copy()
del(allmlf_FilesWordsInDictCopy)
del(UASpeechDataFilesNames)

In [ ]:
print(len(allmlf_FilesWordsInDict)) # labels after update

In [ ]:
convertDictToFile(allmlf_FilesWordsInDict) #convert labels Dict to file after update

# Extract features form waves